<a href="https://colab.research.google.com/github/pedroafleite/data_etl/blob/main/data_etl_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 48.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=d64d32366ac488863fb25ab928fff38c72b00e89171c55b34488c0d1c79ba496
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
Mounted at /content/drive


In [4]:
!pip install flatten_json

  Created wheel for flatten-json: filename=flatten_json-0.1.13-py3-none-any.whl size=7979 sha256=719599d97d998ffcde728a3c42e00dfa825c675a70bdbf9839914b61d524b796
  Stored in directory: /root/.cache/pip/wheels/e6/b3/2a/beb2ceb72d11bf335f9c2f87aae26981f6744f3fc885cde665
Successfully built flatten-json


In [175]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark
import json
from flatten_json import flatten

from pyspark.sql.types import StructType, StructField, ArrayType
from pyspark.sql.types import StringType, DoubleType, BooleanType

from pyspark.sql import Row

In [3]:
spark = SparkSession.builder.getOrCreate()

In [23]:
schema_infomix = StructType([
    StructField("cnpj", StringType(), True),
    StructField("gtin", StringType(), True),
    StructField("category", StringType(), True)
])

In [24]:
infomix = spark.read.option("delimiter", "\t").csv(
    "/content/drive/MyDrive/Colab Notebooks/data_etl/data_etl_test/infomix.tsv", 
    header=True, 
    mode="DROPMALFORMED", 
    schema=schema_infomix
)

In [25]:
infomix.show()

+--------------+--------------+--------------------+
|          cnpj|          gtin|            category|
+--------------+--------------+--------------------+
|08540603000198|06890101502534|     massas e molhos|
|71322150004408|07898605250417|            cervejas|
|43235985000732|07896000724465|              cabelo|
|47508411134004|07896102589122|conservas e enlat...|
|47508411135833|07895000474646|          bomboniere|
|09379531000101|05893202502558|     massas e molhos|
|47508411135833|07891008188271|          bomboniere|
|39346861026802|07891025112983|iogurtes e sobrem...|
|47508411134004|07896262304368|          bomboniere|
|41185455001501|07896085393488|               leite|
|75315333008002|07896036095089|     massas e molhos|
|06057223023112|07898909755533|  doces e sobremesas|
|06057223023112|07896102503708|     massas e molhos|
|75315333008002|07896038308057|               arroz|
|06057223023112|07898027658112|  doces e sobremesas|
|00063960023575|07896005277195|     massas e m

In [186]:
schema_cnpj = StructType([
    StructField("cnpj", StringType(), True),
    StructField("created_at", StringType(), True),
    StructField("id", StringType(), True),
    StructField("response", StructType([
      StructField("abertura", StringType(), True),
      StructField("atividade_principal", ArrayType(StructType([
        StructField("code", StringType(), True),
        StructField("text", StringType(), True)                                                           
      ])), True),
      StructField("atividades_secundarias", ArrayType(StructType([
        StructField("code", StringType(), True),
        StructField("text", StringType(), True)                                          
      ])), True),
      StructField("bairro", StringType(), True),
      StructField("billing", StructType([
        StructField("database", BooleanType(), True),
        StructField("free", BooleanType(), True)                       
      ]), True),
      StructField("capital_social", StringType(), True),
      StructField("cep", StringType(), True),
      StructField("cnpj", StringType(), True),
      StructField("complemento", StringType(), True),
      StructField("data_situacao", StringType(), True),
      StructField("data_situacao_especial", StringType(), True),
      StructField("efr", StringType(), True),
      StructField("email", StringType(), True),
      StructField("fantasia", StringType(), True),
      StructField("google_place_id", StringType(), True),
      StructField("latitude", StringType(), True),
      StructField("logradouro", StringType(), True),
      StructField("longitude", StringType(), True),
      StructField("motivo_situacao", StringType(), True),
      StructField("municipio", StringType(), True),
      StructField("natureza_juridica", StringType(), True),
      StructField("nome", StringType(), True),
      StructField("numero", StringType(), True),
      StructField("qsa", ArrayType(StructType([
        StructField("nome", StringType(), True),
        StructField("qual", StringType(), True)                                          
      ])), True),
      StructField("situacao", StringType(), True),
      StructField("situacao_especial", StringType(), True),
      StructField("status", StringType(), True),
      StructField("telefone", StringType(), True),
      StructField("tipo", StringType(), True),
      StructField("uf", StringType(), True),
      StructField("ultima_atualizacao", StringType(), True)
    ]), True)
  ])

In [108]:
df = spark.createDataFrame(schema=schema_cnpj, data={})

In [109]:
df.printSchema()

root
 |-- cnpj: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- response: struct (nullable = true)
 |    |-- abertura: string (nullable = true)
 |    |-- atividade_principal: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- atividades_secundarias: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- billing: struct (nullable = true)
 |    |    |-- database: boolean (nullable = true)
 |    |    |-- free: boolean (nullable = true)
 |    |-- capital_social: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cnpj: string (nullable = true)
 |    |-- complemento: string (nullable = true)
 |    |-- 

In [110]:
data[6]

{'cnpj': '75315333008274',
 'created_at': '2018-09-26 10:02:18',
 'id': '17266223-0c0b-4d81-bd9f-e240523d9771',
 'response': {'abertura': '12/09/2008',
  'atividade_principal': [{'code': '47.11-3-01',
    'text': 'Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - hipermercados'}],
  'atividades_secundarias': [{'code': '52.12-5-00',
    'text': 'Carga e descarga'}],
  'bairro': 'SAO JOSE',
  'billing': {'database': True, 'free': True},
  'capital_social': '7609482817.90',
  'cep': '63.024-015',
  'cnpj': '75.315.333/0082-74',
  'complemento': '',
  'data_situacao': '12/09/2008',
  'data_situacao_especial': '',
  'efr': '',
  'email': 'tributario@atacadao.com.br',
  'extra': {},
  'fantasia': 'ATACADAO',
  'google_place_id': 'ChIJ9fcRsVuCoQcR-Gud6w_nm7A',
  'latitude': -7.2292715,
  'logradouro': 'AV PADRE CICERO',
  'longitude': -39.3487502,
  'motivo_situacao': '',
  'municipio': 'JUAZEIRO DO NORTE',
  'natureza_juridica': '204-6 - Sociedade Anôni

In [50]:
import json

data = []
with open("/content/drive/MyDrive/Colab Notebooks/data_etl/data_etl_test/cnpjs_receita_federal.jl") as f:
    for line in f:
        data.append(json.loads(line))

In [158]:
sc = pyspark.SparkContext.getOrCreate()

In [176]:
cosmos_raw = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data_etl/data_etl_test/cosmos.jl', lines=True)
cosmos_json = json.loads(cosmos_raw.to_json(orient="records"))
cosmos_flat = (flatten(record, '.') for record in cosmos_json)
myDf = spark.read.json([cosmos_flat])
myDf.show(truncate=False)

Py4JJavaError: ignored

In [173]:
cosmos_flat

<generator object <genexpr> at 0x7f6978d3acd0>

In [180]:
myjson = sc.parallelize(data)

In [188]:
cnpj = spark.createDataFrame(
    data=data,
    schema=schema_cnpj
)

cnpj.show(truncate=False)

+--------------+-------------------+------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [191]:
cnpj.select("cnpj", "response.municipio").show()

+--------------+--------------------+
|          cnpj|           municipio|
+--------------+--------------------+
|68093095000179|             ATIBAIA|
|13492669000190|           SAO PAULO|
|74683392000177|           SAO PAULO|
|12423658000195|             CARPINA|
|00588458000103|          MONTE ALTO|
|61365557000110|SAO SEBASTIAO DO ...|
|75315333008274|   JUAZEIRO DO NORTE|
|33033028000184|            CURITIBA|
|04939545000119|             ATIBAIA|
|46842894000591|        CANDIDO MOTA|
|60879467000185|           SAO PAULO|
|29737368003487|           GUARAREMA|
|04869719000114|            CURITIBA|
|01097783000640|      BELO HORIZONTE|
|08192116000181|           SAO PAULO|
|07175867000127|           SAO PAULO|
|05215210000110|            CURITIBA|
|06940040000108|               SERRA|
|32223965000130|  RIBEIRAO DO PINHAL|
|59557124000115|           SAO PAULO|
+--------------+--------------------+
only showing top 20 rows



In [141]:
data[7]

{'cnpj': '33033028000184',
 'created_at': '2018-08-24 01:25:49',
 'id': '717e6f70-7058-49b3-b1a8-5306e4178157',
 'response': {'abertura': '24/01/1973',
  'atividade_principal': [{'code': '82.99-7-99',
    'text': 'Outras atividades de serviços prestados principalmente às empresas não especificadas anteriormente'}],
  'atividades_secundarias': [{'code': '10.52-0-00',
    'text': 'Fabricação de laticínios'},
   {'code': '10.93-7-01',
    'text': 'Fabricação de produtos derivados do cacau e de chocolates'},
   {'code': '10.93-7-02',
    'text': 'Fabricação de frutas cristalizadas, balas e semelhantes'},
   {'code': '10.99-6-02', 'text': 'Fabricação de pós alimentícios'},
   {'code': '10.99-6-03', 'text': 'Fabricação de fermentos e leveduras'},
   {'code': '10.99-6-99',
    'text': 'Fabricação de outros produtos alimentícios não especificados anteriormente'},
   {'code': '82.11-3-00',
    'text': 'Serviços combinados de escritório e apoio administrativo'},
   {'code': '82.19-9-99',
    'te